In [1]:
import pandas as pd
import pymongo
from ipyparallel import Client

In [2]:
client = pymongo.MongoClient()
db = client.swatford

In [4]:
gene_pmids = pd.DataFrame(list(db.medline.genes.mappings.find({},{"_id":0})))

In [5]:
pmids = gene_pmids.pmid.unique()

In [6]:
rc = Client()

In [7]:
with rc[:].sync_imports():
    import pymongo

importing pymongo on engine(s)


In [8]:
%%px
client = pymongo.MongoClient()
db = client.swatford

In [9]:
for i in rc.ids:
    rc[i].push({"pmids":pmids[(len(pmids)//8+1)*i:(len(pmids)//8+1)*(i+1)].copy(),"gene_pmids":gene_pmids})

In [10]:
def gene_desc_map():
    to_insert = []
    to_insert_an = []
    for i,pmid in enumerate(pmids):
#         for doc in db.medline.descs.mappings.find({"pmid":pmid},{"_id":0,"name":0}):
#             for g in gene_pmids[gene_pmids.pmid==pmid].gene_id:
#                 doc["gene_id"] = g
#                 to_insert.append(doc.copy())
        for doc in db.medline.descs.ancestors.mappings.find({"pmid":pmid},{"_id":0}):
            for g in gene_pmids[gene_pmids.pmid==pmid].gene_id:
                doc["gene_id"] = g
                to_insert_an.append(doc.copy())
#         if len(to_insert)>=25000:
#             db.genes.descs.original.insert_many(to_insert)
#             to_insert = []
        if len(to_insert_an)>=25000:
            db.genes.descs.ancestors.insert_many(to_insert_an)
            to_insert_an = []
        if i%2000==0:
            print(i)
    db.genes.descs.ancestors.insert_many(to_insert) if len(to_insert_an)>=25000 else None
#     db.genes.descs.original.insert_many(to_insert) if len(to_insert)>=25000 else None

In [11]:
ar = rc[:].apply_async(gene_desc_map)

In [ ]:
len(pmids)

In [39]:
i=0
for x in ar.stdout:
    i+=int(x.strip().split("\n")[-1])
print(len(pmids)-i)

14693


In [40]:
ar.stdout

['0\n2000\n4000\n6000\n8000\n10000\n12000\n14000\n16000\n18000\n20000\n22000\n24000\n26000\n28000\n30000\n32000\n34000\n36000\n38000\n40000\n42000\n44000\n46000\n48000\n50000\n52000\n54000\n56000\n58000\n60000\n62000\n64000\n66000\n68000\n70000\n72000\n74000\n76000\n78000\n80000\n82000\n84000\n86000\n88000\n90000\n92000\n94000\n96000\n98000\n100000\n102000\n104000\n106000\n108000\n110000\n',
 '0\n2000\n4000\n6000\n8000\n10000\n12000\n14000\n16000\n18000\n20000\n22000\n24000\n26000\n28000\n30000\n32000\n34000\n36000\n38000\n40000\n42000\n44000\n46000\n48000\n50000\n52000\n54000\n56000\n58000\n60000\n62000\n64000\n66000\n68000\n70000\n72000\n74000\n76000\n78000\n80000\n82000\n84000\n86000\n88000\n90000\n92000\n94000\n96000\n98000\n100000\n102000\n104000\n106000\n108000\n110000\n',
 '0\n2000\n4000\n6000\n8000\n10000\n12000\n14000\n16000\n18000\n20000\n22000\n24000\n26000\n28000\n30000\n32000\n34000\n36000\n38000\n40000\n42000\n44000\n46000\n48000\n50000\n52000\n54000\n56000\n58000\n60000\

In [41]:
ar.completed

[datetime.datetime(2017, 7, 1, 1, 3, 57, 899960, tzinfo=tzlocal()),
 datetime.datetime(2017, 7, 1, 0, 58, 26, 87250, tzinfo=tzlocal()),
 datetime.datetime(2017, 7, 1, 0, 47, 1, 227825, tzinfo=tzlocal()),
 datetime.datetime(2017, 7, 1, 0, 53, 53, 504091, tzinfo=tzlocal()),
 datetime.datetime(2017, 7, 1, 0, 47, 17, 740292, tzinfo=tzlocal()),
 datetime.datetime(2017, 7, 1, 0, 38, 32, 473062, tzinfo=tzlocal()),
 datetime.datetime(2017, 6, 30, 22, 43, 35, 495205, tzinfo=tzlocal()),
 datetime.datetime(2017, 7, 1, 0, 32, 21, 36567, tzinfo=tzlocal())]

In [ ]:
ar.elapsed/3600